In [25]:
import pandas as pd
import os
import glob
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
from six.moves import cPickle as pickle

###data download site http://research.xiaojukeji.com/competition/detail.action?competitionId=DiTech2016 

In [27]:
# load district hashmap first
def loadCluster(path):
    allFiles = [os.path.join(path,f) for f in os.listdir(path)]
    dlist=[]
    col_names=['district_hash','district_id'] 
    for file_ in allFiles:
        df = pd.read_table(file_, sep='\t', names=col_names)
        dlist.append(df)
    df= pd.concat(dlist,ignore_index=True)
    return dict(zip(df.district_hash, df.district_id))


def loadOrders(path):
    allFiles = [os.path.join(path,f) for f in os.listdir(path)]
    dlist=[]
    col_names=['order_id','driver_id','passenger_id','start_district_id',
               'dest_district_hash', 'Price','Time'] 
    #toload=1
    for file_ in allFiles:
        df = pd.read_table(file_, sep='\t',parse_dates=[6], names=col_names)
        df=df.replace({'start_district_id':cluster})
        dlist.append(df)
        #df['time_slot']=[(t.hour*60+t.minute)/10+1 for t in df.Time]
        #df['date'] = [t.year*1E4+t.month*1E2 + t.day for t in df.Time]       
        #just load one file for testing
        #toload=toload+1
        #if toload>2: break
    df= pd.concat(dlist,ignore_index=True)
    df['time_slot']=[(t.hour*60+t.minute)/10+1 for t in df.Time]
    return df
    
def loadWeather(path):
    allFiles = [os.path.join(path,f) for f in os.listdir(path)]
    dlist=[]
    col_names=['Time','Weather','Temperature','PM25'] 
    #toload=1
    for file_ in allFiles:
        df = pd.read_table(file_, sep='\t',parse_dates=[0], names=col_names)
        df['time_slot']=[(t.hour*60+t.minute)/10+1 for t in df.Time]
        dlist.append(df)
        #df['date'] = [t.year*1E4+t.month*1E2 + t.day for t in df.Time]       
        #just load one file for testing
        #toload=toload+1
        #if toload>2: break
    df= pd.concat(dlist,ignore_index=True)
    return df
    
def loadTraffic(path):
    allFiles = [os.path.join(path,f) for f in os.listdir(path)]
    dlist=[]
    col_names=['district_id','lv1','lv2','lv3','lv4','Time'] 
    #toload=1
    def myfun(s):
        return int(s[2:])
    for file_ in allFiles:
        df = pd.read_table(file_, sep='\t',parse_dates=[5], names=col_names,
                           converters={1:myfun,2:myfun,3:myfun,4:myfun})
        df=df.replace({'district_id':cluster})
        dlist.append(df)
        df['time_slot']=[(t.hour*60+t.minute)/10+1 for t in df.Time]
        #df['date'] = [t.year*1E4+t.month*1E2 + t.day for t in df.Time]       
        #just load one file for testing
        #toload=toload+1
        #if toload>2: break
    return pd.concat(dlist,ignore_index=True)




In [28]:
# load cluster map
path="season_1/training_data/cluster_map"
cluster = loadCluster(path)

# load orders data
path="season_1/training_data/order_data"
orders_train=loadOrders(path)


# load traffic data
path="season_1/training_data/traffic_data"
traffic_train=loadTraffic(path)

#load weather data
path="season_1/training_data/weather_data"
weather_train=loadWeather(path)

# remove Jan 1st which is a holiday
orders_train=orders_train[orders_train.Time.dt.date!=datetime.date(2016,1,1)]
traffic_train=traffic_train[traffic_train.Time.dt.date!=datetime.date(2016,1,1)]
weather_train=weather_train[weather_train.Time.dt.date!=datetime.date(2016,1,1)]

#add the week day and weekend 
orders_train['date'] = orders_train['Time'].dt.date
orders_train.drop(['passenger_id', 'Price', 'Time'], axis =1, inplace= True)
orders_train.date = pd.to_datetime(orders_train.date) 

## only keep time_slot 
weather_train['date'] = weather_train['Time'].dt.date
weather_train.drop(['Time'],axis = 1, inplace=True)
weather_train.drop_duplicates(['date', 'time_slot'], take_last = True, inplace=True)

# sort the date and drop Time 
traffic_train.sort(['district_id', 'Time'], inplace = True)
traffic_train['date'] = traffic_train['Time'].dt.date
traffic_train.drop(['Time'], axis = 1, inplace= True)
traffic_train.rename(columns={'district_id':'start_district_id'}, inplace=True)
traffic_train.date = pd.to_datetime(traffic_train.date)

orders_train.shape

/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:31: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:34: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


(8039327, 7)

In [49]:
# combine the data together 
index_col = ['date','time_slot', 'start_district_id']
grouped = orders_train.groupby(index_col , as_index = False)
train = pd.DataFrame()

train['date'] = grouped.count()['date']
train['time_slot'] = grouped.count()['time_slot']
train['start_district_id']=grouped.count()['start_district_id']
train['demand']=grouped.count()['order_id']
train['supply']=grouped.count()['driver_id']
train['dayType'] = [1 if t.isoweekday()>5 else 0 for t in train.date]
train['gap']=train['demand']-train['supply']

In [57]:
#merge data together 
train_data = pd.merge(train, weather_train, on = ['date', 'time_slot'], how = 'left')
train_data = pd.merge(train_data, traffic_train, on = ['date', 'time_slot', 'start_district_id'], how = 'left' )
train.sort(['date', 'time_slot', 'start_district_id']).head(100)
train.shape

/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


(155262, 7)

In [58]:
## add the last three time slot gap as t-1, t-2, t-3 , but need to remove first three time_slot for each district and day
train_data['t-1'] = train_data.gap.shift(1)
train_data['t-2'] = train_data.gap.shift(2)
train_data['t-3'] = train_data.gap.shift(3)

train_data = train_data[(train_data.time_slot != 1) & (train_data.time_slot != 2) & (train_data.time_slot != 3) ]

In [59]:
print(train_data.shape)
train_data.head(100)

(152251, 17)


,date,time_slot,start_district_id,demand,supply,dayType,gap,Weather,Temperature,PM25,lv1,lv2,lv3,lv4,t-1,t-2,t-3
156,2016-01-02,4,1.0,60,58,1,2,2.0,8.0,131.0,1197.0,213.0,67.0,45.0,0.0,0.0,2.0
157,2016-01-02,4,2.0,6,6,1,0,2.0,8.0,131.0,702.0,62.0,13.0,18.0,2.0,0.0,0.0
158,2016-01-02,4,4.0,13,12,1,1,2.0,8.0,131.0,602.0,98.0,24.0,12.0,0.0,2.0,0.0
159,2016-01-02,4,6.0,4,4,1,0,2.0,8.0,131.0,291.0,40.0,14.0,12.0,1.0,0.0,2.0
160,2016-01-02,4,7.0,50,47,1,3,2.0,8.0,131.0,1822.0,258.0,53.0,55.0,0.0,1.0,0.0
161,2016-01-02,4,8.0,86,84,1,2,2.0,8.0,131.0,2505.0,235.0,114.0,74.0,3.0,0.0,1.0
162,2016-01-02,4,9.0,13,8,1,5,2.0,8.0,131.0,437.0,47.0,24.0,11.0,2.0,3.0,0.0
163,2016-01-02,4,10.0,1,0,1,1,2.0,8.0,131.0,168.0,6.0,3.0,6.0,5.0,2.0,3.0
164,2016-01-02,4,11.0,10,8,1,2,2.0,8.0,131.0,734.0,90.0,19.0,10.0,1.0,5.0,2.0
165,2016-01-02,4,12.0,78,61,1,17,2.0,8.0,131.0,1510.0,127.0,43.0,34.0,2.0,1.0,5.0


In [60]:
## use the mean to put back the NA value in the column 
train_data = train_data.fillna({'Weather': train_data.Weather.mean(), 'PM25': train_data.PM25.mean(), 'Temperature': train_data.Temperature.mean(), 
                        'lv1':  train_data.lv1.mean(), 'lv2':  train_data.lv2.mean(), 
                        'lv3': train_data.lv3.mean(), 'lv4': train_data.lv4.mean()})

## normalize the traffic info 
train_data['lv1_pect'] = train_data.lv1/(train_data.lv1+train_data.lv2+train_data.lv3+train_data.lv4) 
train_data['lv2_pect'] = train_data.lv2/(train_data.lv1+train_data.lv2+train_data.lv3+train_data.lv4) 
train_data['lv3_pect'] = train_data.lv3/(train_data.lv1+train_data.lv2+train_data.lv3+train_data.lv4) 
train_data['lv4_pect'] = train_data.lv4/(train_data.lv1+train_data.lv2+train_data.lv3+train_data.lv4) 


##reorder the cols and put gap in the front 
cols = ["gap", "date","time_slot","start_district_id","demand","supply", "dayType", "Weather","Temperature","PM25","lv1","lv2","lv3","lv4","t-1","t-2","t-3","lv1_pect","lv2_pect","lv3_pect","lv4_pect"]
train_data = train_data[cols]



In [61]:
print(train_data.shape)
train_data.head()

(152251, 21)


,gap,date,time_slot,start_district_id,demand,supply,dayType,Weather,Temperature,PM25,...,lv2,lv3,lv4,t-1,t-2,t-3,lv1_pect,lv2_pect,lv3_pect,lv4_pect
156,2,2016-01-02,4,1.0,60,58,1,2.0,8.0,131.0,...,213.0,67.0,45.0,0.0,0.0,2.0,0.786465,0.139947,0.044021,0.029566
157,0,2016-01-02,4,2.0,6,6,1,2.0,8.0,131.0,...,62.0,13.0,18.0,2.0,0.0,0.0,0.883019,0.077987,0.016352,0.022642
158,1,2016-01-02,4,4.0,13,12,1,2.0,8.0,131.0,...,98.0,24.0,12.0,0.0,2.0,0.0,0.817935,0.133152,0.032609,0.016304
159,0,2016-01-02,4,6.0,4,4,1,2.0,8.0,131.0,...,40.0,14.0,12.0,1.0,0.0,2.0,0.815126,0.112045,0.039216,0.033613
160,3,2016-01-02,4,7.0,50,47,1,2.0,8.0,131.0,...,258.0,53.0,55.0,0.0,1.0,0.0,0.832724,0.117916,0.024223,0.025137


In [62]:
##check if there is any data have null value 
train_data.isnull().any()

gap                  False
date                 False
time_slot            False
start_district_id    False
demand               False
supply               False
dayType              False
Weather              False
Temperature          False
PM25                 False
lv1                  False
lv2                  False
lv3                  False
lv4                  False
t-1                  False
t-2                  False
t-3                  False
lv1_pect             False
lv2_pect             False
lv3_pect             False
lv4_pect             False
dtype: bool

In [297]:
####load test data
# load cluster map
path="season_1/test_set_1/cluster_map"
cluster = loadCluster(path)

# load orders data
path="season_1/test_set_1/order_data"
orders_test=loadOrders(path)


# load traffic data
path="season_1/test_set_1/traffic_data"
traffic_test=loadTraffic(path)

#load weather data
path="season_1/test_set_1/weather_data"
weather_test=loadWeather(path)


In [298]:
#add the week day and weekend 
orders_test['date'] = orders_test['Time'].dt.date
orders_test.drop(['passenger_id', 'Price', 'Time'], axis =1, inplace= True)
orders_test.date = pd.to_datetime(orders_test.date)

## only keep time_slot 
weather_test['date'] = weather_test['Time'].dt.date
weather_test.date = pd.to_datetime(weather_test.date)
weather_test.drop(['Time'],axis = 1, inplace=True)
weather_test.drop_duplicates(['date', 'time_slot'], take_last = True, inplace=True)
weather_test.rename(columns={'time_slot': 'time_slot_minus_1'}, inplace=True)

# sort the date and drop Time 
traffic_test.sort(['district_id', 'Time'], inplace = True)
traffic_test['date'] = traffic_test['Time'].dt.date
traffic_test.drop(['Time'], axis = 1, inplace= True)
traffic_test.rename(columns={'time_slot': 'time_slot_minus_1', 'district_id':'start_district_id'}, inplace=True)
traffic_test.date = pd.to_datetime(traffic_test.date)

#group test orders
index_col = ['date','time_slot', 'start_district_id']
grouped =orders_test.groupby(index_col , as_index = False)
test = pd.DataFrame()

test['date'] = grouped.count()['date']
test['dayType'] = [1 if t.isoweekday()>5 else 0 for t in test.date]
test['time_slot'] = grouped.count()['time_slot']
test['start_district_id']=grouped.count()['start_district_id']
test['demand']=grouped.count()['order_id']
test['supply']=grouped.count()['driver_id']
test['gap']=test['demand']-test['supply']

test.date = pd.to_datetime(test.date)
test['start_district_id'] = test['start_district_id'].apply(int)

test.shape

/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


(7924, 7)

In [299]:
predict_time = pd.read_table("season_1/test_set_1/read_me_1.txt",skiprows = 1, sep='-', parse_dates = [[0,1,2]], header=None)
predict_time.columns = ['date', 'time_slot']
predict = pd.concat([predict_time for t in range(1,67)],ignore_index=True)
predict['start_district_id']=[i for i in range(1,67) for j in range(len(predict_time))]
predict['time_slot_minus_1'] = predict['time_slot']-1

print(predict.shape)
predict.head()

(2838, 4)


,date,time_slot,start_district_id,time_slot_minus_1
0,2016-01-22,46,1,45
1,2016-01-22,58,1,57
2,2016-01-22,70,1,69
3,2016-01-22,82,1,81
4,2016-01-22,94,1,93


In [300]:
prepare = pd.merge(predict, test, left_on = ['date', 'start_district_id','time_slot_minus_1'], right_on = ['date', 'start_district_id','time_slot'], how = 'left')

prepare['time_slot'] = prepare['time_slot_x']
prepare['t-1'] = prepare['gap']
prepare.drop(['time_slot_x', 'time_slot_y', 'gap'], axis=1, inplace = True)

In [301]:
for index, row in prepare.iterrows():
    t_2_row = test[(test.date == row.date)&(test.start_district_id == row.start_district_id)&(test.time_slot == row.time_slot-2)]
    t_3_row = test[(test.date == row.date)&(test.start_district_id == row.start_district_id)&(test.time_slot == row.time_slot-3)]
    if(t_2_row.shape[0] != 0):
        prepare.loc[index, 't-2'] = t_2_row.loc[t_2_row.index[0], 'gap']

    if(t_3_row.shape[0] != 0):
        prepare.loc[index, 't-3'] = t_3_row.loc[t_3_row.index[0], 'gap']
    
prepare = prepare.fillna(1)

In [319]:
prepare.shape

(2838, 10)

In [337]:
#merge data together 
test_data = pd.merge(prepare, weather_test, on = ['date', 'time_slot_minus_1'], how = 'left')
test_data = pd.merge(test_data, traffic_test, on = ['date', 'time_slot_minus_1', 'start_district_id'], how = 'left' )

test_data = test_data.fillna({'Weather': test_data.Weather.mean(), 'PM25': test_data.PM25.mean(), 'Temperature': test_data.Temperature.mean(),
                              'lv1': test_data.lv1.mean(), 'lv2': test_data.lv2.mean(), 'lv3': test_data.lv3.mean(), 
                              'lv4': test_data.lv4.mean()})
## normalize the traffic info 
test_data['lv1_pect'] = test_data.lv1/(test_data.lv1+test_data.lv2+test_data.lv3+test_data.lv4) 
test_data['lv2_pect'] = test_data.lv2/(test_data.lv1+test_data.lv2+test_data.lv3+test_data.lv4) 
test_data['lv3_pect'] = test_data.lv3/(test_data.lv1+test_data.lv2+test_data.lv3+test_data.lv4) 
test_data['lv4_pect'] = test_data.lv4/(test_data.lv1+test_data.lv2+test_data.lv3+test_data.lv4) 


In [338]:
test_data.isnull().any()

date                 False
start_district_id    False
time_slot_minus_1    False
dayType              False
demand               False
supply               False
t-1                  False
time_slot            False
t-2                  False
t-3                  False
Weather              False
Temperature          False
PM25                 False
lv1                  False
lv2                  False
lv3                  False
lv4                  False
lv1_pect             False
lv2_pect             False
lv3_pect             False
lv4_pect             False
dtype: bool

In [63]:
pickle_file = 'train_data.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'orders_train': orders_train,
    'weather_train': weather_train, 
    'traffic_train': traffic_train,
    'train_data': train_data,
    'test_data': test_data
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [64]:
## direct read the small test set and training set from the pickle file
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

pickle_file = 'train_data.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  orders_train = save['orders_train']
  weather_train = save['weather_train'] 
  traffic_train = save['traffic_train']
  train_data = save['train_data']
  test_data = save['test_data']
  del save  # hint to help gc free up memory
  print('train_data', train_data.shape)
  print('weather_train', weather_train.shape)
  print('traffic_train', traffic_train.shape)
  print('train_data', train_data.shape)
  print('test_data', test_data.shape)

('train_data', (152251, 21))
('weather_train', (2528, 5))
('traffic_train', (184264, 7))
('train_data', (152251, 21))
('test_data', (2838, 22))
